In [ ]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import albumentations as A
import tqdm
import cv2
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset


batch_size, num_workers = 16, 8
debug = False


resnet_fns = [
    '../input/ranzcr-clip-models/resnet200d_640_1000_fold_0_model-018-0.961305.pth',
    '../input/ranzcr-clip-models/resnet200d_640_1000_fold_1_model-015-0.958331.pth',
    '../input/ranzcr-clip-models/resnet200d_640_1000_fold_2_model-018-0.956810.pth',
    '../input/ranzcr-clip-models/resnet200d_640_1000_fold_3_model-017-0.958175.pth',
    '../input/ranzcr-clip-models/resnet200d_640_1000_fold_4_model-010-0.957087.pth',
]


effnet_fns = [
    '../input/ranzcr-clip-models/tf_efficientnet_b7_ns_1000_fold_0_model-005-0.958068.pth',
    '../input/ranzcr-clip-models/tf_efficientnet_b7_ns_1000_fold_1_model-005-0.955992.pth',
    '../input/ranzcr-clip-models/tf_efficientnet_b7_ns_1000_fold_2_model-004-0.954233.pth',
    '../input/ranzcr-clip-models/tf_efficientnet_b7_ns_1000_fold_3_model-009-0.957107.pth',
    '../input/ranzcr-clip-models/tf_efficientnet_b7_ns_1000_fold_4_model-008-0.955761.pth',
]


class RANZCRDatasetInference(Dataset):
    def __init__(self, df, image_size):
        self.df = df
        self.transform = A.Compose([
            A.Resize(image_size, image_size),
            A.Normalize(
                mean=(0.485),
                std=(0.229),
            ),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.loc[index]
        img = cv2.imread(row.file_path, cv2.IMREAD_GRAYSCALE)
        mask = img > 0
        img = img[np.ix_(mask.any(1), mask.any(0))]

        img = np.expand_dims(img, -1)

        res = self.transform(image=img)

        img = res['image']
        img = torch.from_numpy(img.transpose(2, 0, 1))

        return img


def inference(models, test_loader, device='cuda'):
    probs = []
    for images in tqdm.tqdm(test_loader):
        images = images.to(device)
        y_preds = []
        for model in models:
            batch_preds = []
            for tta in [lambda x: x, lambda x: x.flip(-1)]:
                with torch.no_grad():
                    batch_preds.append(model(tta(images)).sigmoid().to('cpu').numpy())
            batch_preds = np.mean(batch_preds, axis=0)
            y_preds.append(batch_preds)
        y_preds = np.mean(y_preds, axis=0)
        probs.append(y_preds)
    probs = np.concatenate(probs)
    return probs

In [ ]:
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')
test['file_path'] = test.StudyInstanceUID.apply(lambda x: os.path.join('../input/ranzcr-clip-catheter-line-classification/test', f'{x}.jpg'))

In [ ]:
models = []

for fn in effnet_fns:
    model = torch.load(fn, map_location={'cuda:1':'cuda:0'})
    model.eval()
    models.append(model)

test_dataset = RANZCRDatasetInference(test, 1000)
if debug:
    test_dataset = Subset(test_dataset, list(range(batch_size)))
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True)

effnet_predictions = inference(models, test_loader)

In [ ]:
models = []

for fn in resnet_fns:
    model = torch.load(fn, map_location={'cuda:1':'cuda:0'})
    model.eval()
    models.append(model)

test_dataset = RANZCRDatasetInference(test, 640)
if debug:
    test_dataset = Subset(test_dataset, list(range(batch_size)))
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True)

resnet_predictions = inference(models, test_loader)

In [ ]:
predictions = (effnet_predictions * 0.971 + resnet_predictions * 0.965) / (0.971 + 0.965)

test = test.drop('file_path', axis=1)

if debug:
    test.iloc[:batch_size, 1:] = predictions
else:
    test.iloc[:, 1:] = predictions

test.to_csv('submission.csv', index=None)